### Project: Data Modeling with Apache Cassandra

> Andrew Nachitgal  
> Data Engineering Nanodegree  
> Project 2


# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [3]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [4]:
## checking your current working directory
# print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath, topdown=False):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

print(file_path_list)

['/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-27-events.csv', '/home/work

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [6]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Create and connect to Apache Cassandra database

### The CSV file titled 'event_datafile_new.csv' contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

## Create a Cassandra database connection, create a keyspace, and connect to the keyspace

#### Creating a Cluster

In [7]:
# Make a connection to a Cassandra instance on your local machine (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# Establish connection and begin executing queries
session = cluster.connect()

#### Create Keyspace

In [21]:
# Create Keyspace 
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

#### Set Keyspace

In [9]:
# Set KEYSPACE
session.set_keyspace('sparkify')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## This project creates databases to answer the following three queries:

1. Return artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4.    
2. Return the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182.  
3. Return every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'.  





### Query 1. Return artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4.

* PRIMARY KEY:  
This is a compound key consisting of a PARTITION by sessionId with CLUSTERING on the itemInSession column, which is appropriate for returning results from a query for records from a specific item in a session of a particular session Id.

* Table Data:  
The data inserted into the tables are sessionId and itemInSession for the PRIMARY KEY, with artist name, song title, and song length to be returned.

#### Create and Load Table

In [10]:
query = "CREATE TABLE IF NOT EXISTS song_session_history"
query = query + "(artist_name text, \
                    song_title text, \
                    song_length float, \
                    sessionId int, \
                    itemInSession int, \
                    PRIMARY KEY (sessionId, itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable        
        query = "INSERT INTO song_session_history (artist_name, song_title, song_length, sessionId, itemInSession)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

#### Run a SELECT query to verify that the data have been inserted into each table

In [12]:
query = "SELECT artist_name, song_title, song_length FROM song_session_history WHERE sessionId = 338 AND itemInSession = 4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query 2. Return the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182.

* PRIMARY KEY:  
This is a compound key consisting of a PARTITION by userId with CLUSTERING on the sessionId, userId, and itemInSession columns, which is appropriate for returning results from a query for records from a specific session Id of a particular user, with the return records sorted by session position.

* Table Data:  
The data inserted into the tables are userId, sessionId, and itemInSession for the PRIMARY KEY, with artist name, song title, user first name, and user last name to be returned.

#### Create and Load Table

In [13]:
query = "CREATE TABLE IF NOT EXISTS user_session_history"
query = query + "(artist_name text, \
                    song_title text, \
                    user_first_name text, \
                    user_last_name text, \
                    sessionId int, \
                    userId int, \
                    itemInSession int, \
                    PRIMARY KEY (sessionId, userId, itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable        
        query = "INSERT INTO user_session_history (artist_name, song_title, user_first_name, user_last_name, sessionId, userid, itemInSession)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], line[1], line[4], int(line[8]), int(line[10]), int(line[3])))

#### Run a SELECT query to verify that the data have been inserted into each table.

In [15]:
query = "SELECT artist_name, song_title, user_first_name, user_last_name FROM user_session_history WHERE sessionId = 182 AND userId = 10"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.user_first_name, row.user_last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query 3. Return every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'.

* PRIMARY KEY:  
This is a compound key consisting of a PARTITION by song with CLUSTERING on the userId column, which is appropriate for returning results from a query for a list of users who have listened to a particlar song. The userId colum allows clustering to distinguish users with the same name.

* Table Data:  
The data inserted into the tables are song and userId for the PRIMARY KEY, with user first and last name to be returned.

#### Create and Load Table

In [18]:
query = "CREATE TABLE IF NOT EXISTS user_song_history "
query = query + "(song_title text, \
                    user_first_name text, \
                    user_last_name text, \
                    userId int, \
                    PRIMARY KEY (song_title, userId))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [19]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable        
        query = "INSERT INTO user_song_history (song_title, user_first_name, user_last_name, userId)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], line[1], line[4], int(line[10])))

#### Run a SELECT to verify that the data have been inserted into each table.

In [20]:
query = "SELECT user_first_name, user_last_name FROM user_song_history WHERE song_title = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user_first_name, row.user_last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
## Drop the table before closing out the sessions
query = "DROP TABLE song_session_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query2 = "DROP TABLE user_session_history"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
query3 = "DROP TABLE user_song_history"
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()